In [29]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.metrics import f1_score

train_x,test_x = [],[]
for feat in sorted(glob.glob('../feature/2_level_stacking_pkl/*')):
    if 'fm2.pkl' in feat or 'ligbm' in  feat :
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
    

train = pd.read_csv("../input/new_data/train_set.csv")
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)

file path ../feature/2_level_stacking_pkl/2leve_lgb_stacking.pkl
(102277, 19) (102277, 19)
file path ../feature/2_level_stacking_pkl/2leve_lr_stacking.pkl
(102277, 19) (102277, 19)
file path ../feature/2_level_stacking_pkl/2leve_nn_stacking.pkl
(102277, 19) (102277, 19)
file path ../feature/2_level_stacking_pkl/2leve_nn_stackingml.pkl
(102277, 19) (102277, 19)
(102277, 76)


In [30]:
y=(train["class"]-1).astype(int)
from keras.utils import np_utils
train_y=np_utils.to_categorical(y,num_classes=19)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [31]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [32]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss'):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred, val_pred = np.zeros((102277,20)), np.zeros((102277,20))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_f1_l,all_val_f1_l = 0,0
    
    for i in range(19):
        val_loss_l,train_loss_l = 0,0
        val_f1_l,train_f1_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x
            # share params
            params = {
                    'application': 'binary',
                    #'num_leaves': 8,
                    #'lambda_l1': 1,
                    'lambda_l2': 1.0,
                    'max_depth': 4,
                    #'scale_pos_weight':0.9,
                    'metric': met, # or auc
                    'data_random_seed': 2,
                    'learning_rate':lr,
                    # 'bagging_fraction': bagging_fraction,
                    # 'bagging_freq':bag_frec,
                    'feature_fraction': feature_fraction,

                    }
            if met == 'auc':
                s_round = 100
            else:
                s_round = 50
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            train_pred = model.predict(curr_x)
            val_pred[test_index, i] = model.predict(hold_out_x)
            curr_train_loss = log_loss(curr_y[:,i],train_pred)
            curr_val_loss = log_loss(hold_out_y[:,i],val_pred[test_index, i])

            curr_train_f1 = self_f1_score(curr_y[:,i],train_pred)
            curr_val_f1 = self_f1_score(hold_out_y[:,i],val_pred[test_index, i])

            print('ls',curr_train_loss,curr_val_loss,'f1',curr_train_f1,curr_val_f1)
            val_loss_l += curr_val_loss
            train_loss_l += curr_train_loss
            val_f1_l += curr_val_f1
            train_f1_l += curr_train_f1
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_f1_l = train_f1_l/k
        val_f1_l = val_f1_l/k
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class f1 train',train_f1_l,'f1 val',val_f1_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/19
        all_val_loss_l += val_loss_l/19
        all_train_f1_l += train_f1_l/19
        all_val_f1_l += val_f1_l/19
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all f1 avg',all_train_f1_l,all_val_f1_l)
    print('=======================================================')
    return val_pred, test_pred

print('done')

done


In [33]:
lgb_train, lgb_res = simple_ens('lgb',10,233,0.05,0.6)

/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:186: UserWarning: Usage subset(sliced data) of np.ndarray is not recommended due to it will double the peak memory cost in LightGBM.
  warnings.warn("Usage subset(sliced data) of np.ndarray is not recommended due to it will double the peak memory cost in LightGBM.")


0 fold: ls 0.08880606520405047 0.09576106762535988 f1 0.8051755615061622 0.803217730009248
1 fold: ls 0.09427630743150099 0.09896217279710383 f1 0.7997127117335321 0.7953951939305917
2 fold: ls 0.09419485651511902 0.10272405643904228 f1 0.8000089271470618 0.783342000393225
3 fold: ls 0.09453740704211325 0.10042009086771904 f1 0.7987187793495825 0.7923182794174546
4 fold: ls 0.09541127787180824 0.09694931907573348 f1 0.7989445893813214 0.8005089720251142
5 fold: ls 0.09180329083480955 0.10118744758618112 f1 0.8019971840889325 0.8022244875970728
6 fold: ls 0.09456499194000459 0.1035045996344501 f1 0.800647618014901 0.7883464376920093
7 fold: ls 0.09225123750416525 0.09868609124353649 f1 0.8031472541068991 0.7973597732967104
8 fold: ls 0.09363118082803158 0.09995984128400906 f1 0.8002055755580528 0.7859364865202989
9 fold: ls 0.0947756895780983 0.09957096728458081 f1 0.7999675557366135 0.7928527503958323
this class avg train 0.09342523047497013 avg val 0.0997725653837716
this class f1 tra

7 fold: ls 0.0756174323713398 0.08393770711074289 f1 0.8744644199346419 0.8656691601659485
8 fold: ls 0.07432193056315904 0.0883884360400981 f1 0.8769112079088492 0.8571745470362013
9 fold: ls 0.07638429487886236 0.0816082520313408 f1 0.8753092104505411 0.8656355607459582
this class avg train 0.07614738121944464 avg val 0.08189918755724254
this class f1 train 0.8746668512693816 f1 val 0.8693815346055473
0 fold: ls 0.023294266569339 0.026566737075096186 f1 0.9729754554077596 0.9701991602831441
1 fold: ls 0.02322344057811677 0.026470469421864845 f1 0.9735823242383443 0.9704677263073691
2 fold: ls 0.023390420846695403 0.02580229346742035 f1 0.9733957753520324 0.9688795900032308
3 fold: ls 0.02077548003578063 0.02210651866323882 f1 0.9751269063613491 0.9778069142463892
4 fold: ls 0.023087994800617705 0.02909363149097814 f1 0.9734446207950967 0.9654253637194776
5 fold: ls 0.02282866061616694 0.03596358095041251 f1 0.9739992833111033 0.9649630979282262
6 fold: ls 0.022759191629129448 0.02623

3 fold: ls 0.021575531585384083 0.02408357974569117 f1 0.928411911739478 0.9267518771960034
4 fold: ls 0.02065684697851248 0.025973123092035956 f1 0.9317796112880041 0.9073252312952187
5 fold: ls 0.02124120711040182 0.02414602291180266 f1 0.9292501312779567 0.9248947701801167
6 fold: ls 0.021984951375978317 0.026250473056379914 f1 0.9276675765582919 0.9107278603170086
7 fold: ls 0.021580945005048396 0.02658783874055033 f1 0.928116279844902 0.9180496544112356
8 fold: ls 0.0203423485295453 0.02287177734979841 f1 0.9311356292354694 0.9290293479358029
9 fold: ls 0.021959891294289133 0.026860113821662413 f1 0.9287888523834409 0.9059722076284544
this class avg train 0.021182521210005828 avg val 0.026351862058600878
this class f1 train 0.9298274544772485 f1 val 0.915400575334583
0 fold: ls 0.032665216804624614 0.03923999458949494 f1 0.8913383731764049 0.8830370873657696
1 fold: ls 0.03230475927772502 0.03309626070819445 f1 0.8915833314781665 0.8882099774139494
2 fold: ls 0.031580607661532374 